In [1]:
# This pandas file merges two financial spreadsheets and one disaster summary spreadsheet.

# The column that facilitates the merge is a concatenation of disaster number, state and county.

# The resulting dataframe can track financial damage by disaster type.

In [2]:
# We'll use Pandas and Numpy for this Program
import pandas as pd
import numpy as np
import requests
import xmltodict, json
from pandas.io.json import json_normalize



In [3]:
# Read the three necessary files. The first is a summary of declared disasters
# dating back to 1953, with no financial data.  The next two describe FEMA 
# approved moneys for owners and renters, respective, in FY 2019.
# It also defines a url that will allow us to access zillows

df1 = pd.read_csv('./start_data/DisasterDeclarationsSummaries.csv')
df2 = pd.read_csv('./start_data/FEMA_Housing_Assistance_Owners.csv')
df3 = pd.read_csv('./start_data/FEMA_Housing_Assistance_Renters.csv')
url4="http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=X1-ZWz17giky9jf9n_43hp8&state=tx&childtype=county"

In [4]:
# Before we begin cleaning the dataframes, we need to create a unique key where the 
# three dataframes can merge.  We will combine the disaster number, the state, and the county
# to create a row called "address" for all three columns.  TO do this, we have to convert the 
# disaster number into a string.  Then we can concatenate.  Then we will turn the disaster 
# number back into a number for later data processing.

df1["disasterNumber"] = df1["disasterNumber"].astype(str)
df1["address"] = df1["disasterNumber"] + ', ' + df1["state"] + ', ' + df1["designatedArea"]
df1["disasterNumber"] = df1["disasterNumber"].astype(int)

df2["Disaster"] = df2["Disaster"].astype(str)
df2["address"] = df2["Disaster"] + ', ' + df2["State"] + ', ' + df2["County"]
df2["Disaster"] = df2["Disaster"].astype(int)

df3["Disaster"] = df3["Disaster"].astype(str)
df3["address"] = df3["Disaster"] + ', ' + df3["State"] + ', ' + df3["County"]
df3["Disaster"] = df3["Disaster"].astype(int)


In [5]:
# We're going to prepare the two financial dataframes first.  
# Then we'll merge the financial dataframes.
# Then we'll prepare the disasters summary dataframe
# Then we'll merge the disaster summary to the merged financial df.

In [6]:
# First, we'll change the columns we need so the data is labeled as Owners.

df2 = df2.rename(columns ={"Total Approved IHP Amount" : "IHP Amount for Owners"})
df2 = df2.rename(columns ={"Approved for FEMA Assistance" : "Owners Approved"})
df2 = df2.rename(columns ={"Approved between $1 and $10,000" : "Owners <$10k"})
df2 = df2.rename(columns ={"Approved between $10,001 and $25,000" : "Owners $10-25k"})
df2 = df2.rename(columns ={"Approved between $25,001 and Max" : "Owners $25k - Max"})
df2 = df2.rename(columns ={"Total Max Grants" : "Owners Max"})

# Print out the names of columns to verify

df2.columns     

Index(['Disaster', 'State', 'County', 'City', 'Zip Code',
       'Valid Registrations', 'Average FEMA Inspected Damage',
       'Total Inspected', 'Total Damage', 'No FEMA Inspected Damage',
       'FEMA Inspected Damage between $1 and $10,000',
       'FEMA Inspected Damage between $10,001 and $20,000',
       'FEMA Inspected Damage between $20,001 and $30,000',
       'FEMA Inspected Damage > $30,000', 'Owners Approved',
       'IHP Amount for Owners', 'Repair/Replace Amount', 'Rental Amount',
       'Other Needs Amount', 'Owners <$10k', 'Owners $10-25k',
       'Owners $25k - Max', 'Owners Max', 'address'],
      dtype='object')

In [7]:
# Next, we create a new Owners dataframe with select columns from the imported df.

df2a = df2.loc[:,["address", "Disaster", "IHP Amount for Owners","Owners Approved", 
                 "Owners <$10k", "Owners $10-25k", "Owners $25k - Max", "Owners Max"]]

# Resort the data according to the disaster number, then get rid of the disaster information.
df2a = df2a.sort_values("Disaster")
del(df2a["Disaster"])

# Due to the dollar signs and commas, this data frame is being read as a string. 
# These non-number characters will be deleted, and the data can be expressed as a number.

df2a['IHP Amount for Owners'] = df2a['IHP Amount for Owners'].str.replace(',','').str.replace('$','').astype('int')
df2a['Owners Approved'] = df2a['Owners Approved'].str.replace(',','').str.replace('$','').astype('int')
df2a['Owners <$10k'] = df2a['Owners <$10k'].str.replace(',','').str.replace('$','').astype('int')
df2a['Owners $10-25k'] = df2a['Owners $10-25k'].str.replace(',','').str.replace('$','').astype('int')
df2a['Owners $25k - Max'] = df2a['Owners $25k - Max'].str.replace(',','').str.replace('$','').astype('int')
df2a['Owners Max'] = df2a['Owners Max'].str.replace(',','').str.replace('$','').astype('int')

# Check that all the data columns are numbers.
df2a.dtypes

address                  object
IHP Amount for Owners     int32
Owners Approved           int32
Owners <$10k              int32
Owners $10-25k            int32
Owners $25k - Max         int32
Owners Max                int32
dtype: object

In [8]:
# Display the Owners dataframe

df2a

# There are many duplicate rows because in the original file, each row is a distinct zipcode.
# There are many zipcodes in a county.  However, our Disaster Declarations spreadsheet only
# counts counties, not zipcodes, so we will set this spreadsheet up by county as well.

,address,IHP Amount for Owners,Owners Approved,Owners <$10k,Owners $10-25k,Owners $25k - Max,Owners Max
25826,"4116, IL, Peoria (County)",1780,3,3,0,0,0
25335,"4116, IL, Cook (County)",140108,90,90,0,0,0
25334,"4116, IL, Cook (County)",1303082,598,576,21,1,0
25333,"4116, IL, Cook (County)",3146237,1553,1528,25,0,0
25332,"4116, IL, Cook (County)",12012,3,3,0,0,0
...,...,...,...,...,...,...,...
26887,"4466, TX, Harris (County)",0,0,0,0,0,0
26886,"4466, TX, Harris (County)",0,0,0,0,0,0
26885,"4466, TX, Harris (County)",136148,25,21,4,0,0
26913,"4466, TX, Harris (County)",51222,14,13,1,0,0


In [9]:
#Combine the Owners data by the address and sum up all information.

df2b = df2a.groupby(["address"])
df2b = df2b.sum()

# Check to see if the data looks good.  The Owners Spreadsheet is done.
df2b

,IHP Amount for Owners,Owners Approved,Owners <$10k,Owners $10-25k,Owners $25k - Max,Owners Max
address,,,,,,
"4116, IL, Brown (County)",32272,8,6,2,0,0
"4116, IL, Bureau (County)",622024,260,251,9,0,0
"4116, IL, Calhoun (County)",31821,14,14,0,0,0
"4116, IL, Clark (County)",44331,11,9,2,0,0
"4116, IL, Cook (County)",90293298,36895,35846,966,83,29
...,...,...,...,...,...,...
"4466, TX, Jefferson (County)",18405207,2319,1436,842,41,0
"4466, TX, Liberty (County)",3696788,579,435,137,7,0
"4466, TX, Montgomery (County)",8022615,1100,780,293,27,0


In [10]:
# df3 is the Renters' data which is treated just like the Owners data.
# Rename the columns so they say "renters"

df3 = df3.rename(columns ={"Total Approved IHP Amount" : "IHP Amount for Renters"})
df3 = df3.rename(columns ={"Approved for FEMA Assistance" : "Renters Approved"})
df3 = df3.rename(columns ={"Approved between $1 and $10,000" : "Renters <$10k"})
df3 = df3.rename(columns ={"Approved between $10,001 and $25,000" : "Renters $10-25k"})
df3 = df3.rename(columns ={"Approved between $25,001 and Max" : "Renters $25k - Max"})
df3 = df3.rename(columns ={"Total Max Grants" : "Renters Max"})

In [11]:
# Create a new Renters dataframe with only the columns you want

df3a = df3.loc[:,["address", "Disaster", "IHP Amount for Renters","Renters Approved", 
                 "Renters <$10k", "Renters $10-25k", "Renters $25k - Max", "Renters Max"]]

# The ol' sort-and-short
df3a = df3a.sort_values("Disaster")
del(df3a["Disaster"])

# This time only the IHP column is made of strings.  Make them numbers.
df3a['IHP Amount for Renters'] = df3a['IHP Amount for Renters'].str.replace(',','').str.replace('$','').astype('int')

# Make sure they're all numbers
df3a.dtypes

address                   object
IHP Amount for Renters     int32
Renters Approved           int64
Renters <$10k             object
Renters $10-25k           object
Renters $25k - Max         int64
Renters Max                int64
dtype: object

In [12]:
# Group the Renters data so they have the same address and sum up the values that get grouped
df3b = df3a.groupby(["address"])
df3b = df3b.sum()

df3b

,IHP Amount for Renters,Renters Approved,Renters $25k - Max,Renters Max
address,,,,
"4116, IL, Bureau (County)",29177,19,0,0
"4116, IL, Calhoun (County)",7079,4,0,0
"4116, IL, Clark (County)",0,0,0,0
"4116, IL, Cook (County)",37099655,12965,35,1
"4116, IL, Crawford (County)",2079,1,0,0
...,...,...,...,...
"4466, TX, Jefferson (County)",3933428,1207,0,0
"4466, TX, Liberty (County)",566726,184,0,0
"4466, TX, Montgomery (County)",813403,207,0,0


In [13]:
# We can now merge the two data frames.  

merge23 = pd.merge(df2b, df3b, how = 'outer', on=['address'])
merge23

# We see some 'NaN's, which suggests that some counties were represented in the Owners
# spreadsheet but not renters and vice versa. The owners df had 1149 rows and the 
# renters had 1117 rows.  Our current df is 1160, which confirms the discrepancies.

,IHP Amount for Owners,Owners Approved,Owners <$10k,Owners $10-25k,Owners $25k - Max,Owners Max,IHP Amount for Renters,Renters Approved,Renters $25k - Max,Renters Max
address,,,,,,,,,,
"4116, IL, Brown (County)",32272.0,8.0,6.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN
"4116, IL, Bureau (County)",622024.0,260.0,251.0,9.0,0.0,0.0,29177.0,19.0,0.0,0.0
"4116, IL, Calhoun (County)",31821.0,14.0,14.0,0.0,0.0,0.0,7079.0,4.0,0.0,0.0
"4116, IL, Clark (County)",44331.0,11.0,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
"4116, IL, Cook (County)",90293298.0,36895.0,35846.0,966.0,83.0,29.0,37099655.0,12965.0,35.0,1.0
...,...,...,...,...,...,...,...,...,...,...
"4337, FL, San Juan (Municipio)",NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
"4337, FL, Statewide",NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
"4393, NC, Greenwood (County)",NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [14]:
# As a final step, we'll change the NaN's to zeroes.

merge23 = merge23.fillna(value = '0')
merge23

# This is the merged financial data. There are some empty rows here we can delete, but we can take care of those later.

,IHP Amount for Owners,Owners Approved,Owners <$10k,Owners $10-25k,Owners $25k - Max,Owners Max,IHP Amount for Renters,Renters Approved,Renters $25k - Max,Renters Max
address,,,,,,,,,,
"4116, IL, Brown (County)",32272,8,6,2,0,0,0,0,0,0
"4116, IL, Bureau (County)",622024,260,251,9,0,0,29177,19,0,0
"4116, IL, Calhoun (County)",31821,14,14,0,0,0,7079,4,0,0
"4116, IL, Clark (County)",44331,11,9,2,0,0,0,0,0,0
"4116, IL, Cook (County)",9.02933e+07,36895,35846,966,83,29,3.70997e+07,12965,35,1
...,...,...,...,...,...,...,...,...,...,...
"4337, FL, San Juan (Municipio)",0,0,0,0,0,0,0,0,0,0
"4337, FL, Statewide",0,0,0,0,0,0,0,0,0,0
"4393, NC, Greenwood (County)",0,0,0,0,0,0,0,0,0,0


In [15]:
# We'll now clean up the Disaster Summaries data.  First, let's find out what our columns are

print(df1.columns)



Index(['femaDeclarationString', 'disasterNumber', 'state', 'declarationType',
       'declarationDate', 'fyDeclared', 'incidentType', 'declarationTitle',
       'ihProgramDeclared', 'iaProgramDeclared', 'paProgramDeclared',
       'hmProgramDeclared', 'incidentBeginDate', 'incidentEndDate',
       'disasterCloseoutDate', 'fipsStateCode', 'fipsCountyCode', 'placeCode',
       'designatedArea', 'declarationRequestNumber', 'hash', 'lastRefresh',
       'address'],
      dtype='object')


In [16]:
# Let's take the 'address', 'disasterNumber', 'incidentType' and 'fyDeclared' 
# Our financial spreadsheets only range from disaster 4116 to 4466, so we'll trim the other disasters.

df1a = df1.loc[:,["address", "disasterNumber", "incidentType", "fyDeclared"]]
df1a = df1a.loc[(df1 ["disasterNumber"]>=4116), :]
df1a = df1a.loc[(df1 ["disasterNumber"]<=4466), :] 

# Give it the ol' sort-and-short and see what we have.
df1a = df1a.sort_values("disasterNumber")
del(df1a['disasterNumber'])
df1a

# 5,614 columns?! That's 5 times the amount we have in the financial data. Clearly we need to do some trimming.

,address,incidentType,fyDeclared
44632,"4116, IL, Cook (County)",Flood,2013
44764,"4116, IL, Fulton (County)",Flood,2013
44762,"4116, IL, Lake (County)",Flood,2013
44758,"4116, IL, Monroe (County)",Flood,2013
44744,"4116, IL, McDonough (County)",Flood,2013
...,...,...,...
50990,"4466, TX, Orange (County)",Flood,2020
51045,"4466, TX, San Jacinto (County)",Flood,2020
50991,"4466, TX, Chambers (County)",Flood,2020
50992,"4466, TX, Jefferson (County)",Flood,2020


In [17]:
# In order to merge, we have to group the data by "address" because that how the other dfs are set up.
# Each grouping will have one and only one entry.  Therefore, we can get away with using the command
# groupby_df.first()  

df1b = df1a.groupby("address")
df1b = df1b.first()
df1b

,incidentType,fyDeclared
address,,
"4116, IL, Adams (County)",Flood,2013
"4116, IL, Brown (County)",Flood,2013
"4116, IL, Bureau (County)",Flood,2013
"4116, IL, Calhoun (County)",Flood,2013
"4116, IL, Carroll (County)",Flood,2013
...,...,...
"4466, TX, Liberty (County)",Flood,2020
"4466, TX, Montgomery (County)",Flood,2020
"4466, TX, Orange (County)",Flood,2020


In [18]:
# Now we can merge the data into one big sheet

merge_all = pd.merge(merge23, df1b, how = 'outer', on=['address'])
merge_all

,IHP Amount for Owners,Owners Approved,Owners <$10k,Owners $10-25k,Owners $25k - Max,Owners Max,IHP Amount for Renters,Renters Approved,Renters $25k - Max,Renters Max,incidentType,fyDeclared
address,,,,,,,,,,,,
"4116, IL, Brown (County)",32272,8,6,2,0,0,0,0,0,0,Flood,2013.0
"4116, IL, Bureau (County)",622024,260,251,9,0,0,29177,19,0,0,Flood,2013.0
"4116, IL, Calhoun (County)",31821,14,14,0,0,0,7079,4,0,0,Flood,2013.0
"4116, IL, Clark (County)",44331,11,9,2,0,0,0,0,0,0,Flood,2013.0
"4116, IL, Cook (County)",9.02933e+07,36895,35846,966,83,29,3.70997e+07,12965,35,1,Flood,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...
"4465, NC, Sampson (County)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hurricane,2020.0
"4465, NC, Tyrrell (County)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hurricane,2020.0
"4465, NC, Washington (County)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hurricane,2020.0


In [19]:
# Most of these cells are NaN, which means all of the columns have become strings.
# Fill the NaN's as zeroes and convert the two IHP values as integers.

merge_all = merge_all.fillna(value = '0')
merge_all['IHP Amount for Owners'] = merge_all['IHP Amount for Owners'].astype('int')
merge_all['IHP Amount for Renters'] = merge_all['IHP Amount for Renters'].astype('int')

# Confirm the data types
merge_all.dtypes


IHP Amount for Owners      int32
Owners Approved           object
Owners <$10k              object
Owners $10-25k            object
Owners $25k - Max         object
Owners Max                object
IHP Amount for Renters     int32
Renters Approved          object
Renters $25k - Max        object
Renters Max               object
incidentType              object
fyDeclared                object
dtype: object

In [20]:
# We're going to define "relevant" counties that were provided money.
# We trim any data where the total IHP amount totaled zero.

merge_all = merge_all.loc[(merge_all["IHP Amount for Owners"] + merge_all["IHP Amount for Renters"] > 0), :]

merge_all

,IHP Amount for Owners,Owners Approved,Owners <$10k,Owners $10-25k,Owners $25k - Max,Owners Max,IHP Amount for Renters,Renters Approved,Renters $25k - Max,Renters Max,incidentType,fyDeclared
address,,,,,,,,,,,,
"4116, IL, Brown (County)",32272,8,6,2,0,0,0,0,0,0,Flood,2013
"4116, IL, Bureau (County)",622024,260,251,9,0,0,29177,19,0,0,Flood,2013
"4116, IL, Calhoun (County)",31821,14,14,0,0,0,7079,4,0,0,Flood,2013
"4116, IL, Clark (County)",44331,11,9,2,0,0,0,0,0,0,Flood,2013
"4116, IL, Cook (County)",90293298,36895,35846,966,83,29,37099655,12965,35,1,Flood,2013
...,...,...,...,...,...,...,...,...,...,...,...,...
"4466, TX, Montgomery (County)",8022615,1100,780,293,27,0,813403,207,0,0,Flood,2020
"4466, TX, Orange (County)",9672865,1585,1186,374,25,0,1039349,322,0,0,Flood,2020
"4466, TX, San Jacinto (County)",82351,20,18,2,0,0,4214,3,0,0,Flood,2020


In [21]:
# Export the file
output = './interim_data/Merged_data.csv'
merge_all.to_csv(output,index = True)

In [22]:
DDa_df = df1.loc[:,["incidentType", "fyDeclared", "address"]]

DDa_df = DDa_df.groupby(["incidentType", "fyDeclared"]).count()
# Convert to DataFrame
DDa_df = DDa_df.reset_index()
# Preview DataFrame
incidents_df = DDa_df.rename(columns = {"address": "counties affected"})

incidents_df

,incidentType,fyDeclared,counties affected
0,Chemical,2014,9
1,Coastal Storm,1972,15
2,Coastal Storm,1975,1
3,Coastal Storm,1980,3
4,Coastal Storm,1983,40
...,...,...,...
463,Volcano,1955,1
464,Volcano,1980,47
465,Volcano,1990,1
466,Volcano,2015,1


In [23]:
inc_piv_df = incidents_df.pivot(index = 'fyDeclared', columns = 'incidentType', values = 'counties affected')
inc_piv_df = inc_piv_df.fillna(value = 0)
inc_piv_df.tail(5)

incidentType,Chemical,Coastal Storm,Dam/Levee Break,Drought,Earthquake,Fire,Fishing Losses,Flood,Freezing,Human Cause,...,Other,Severe Ice Storm,Severe Storm(s),Snow,Terrorist,Tornado,Toxic Substances,Tsunami,Typhoon,Volcano
fyDeclared,,,,,,,,,,,,,,,,,,,,,
2016,0.0,1.0,0.0,0.0,0.0,52.0,0.0,400.0,0.0,0.0,...,0.0,21.0,393.0,74.0,0.0,1.0,1.0,0.0,0.0,0.0
2017,0.0,0.0,3.0,0.0,0.0,90.0,0.0,220.0,0.0,0.0,...,0.0,20.0,257.0,55.0,0.0,45.0,0.0,0.0,0.0,0.0
2018,0.0,3.0,0.0,0.0,0.0,105.0,0.0,185.0,0.0,0.0,...,0.0,0.0,121.0,39.0,0.0,2.0,0.0,0.0,7.0,1.0
2019,0.0,42.0,0.0,0.0,8.0,24.0,0.0,648.0,0.0,0.0,...,0.0,0.0,554.0,0.0,0.0,13.0,0.0,0.0,9.0,0.0
2020,0.0,0.0,0.0,0.0,0.0,14.0,0.0,8.0,0.0,0.0,...,0.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0


In [24]:
output_data_file = "./interim_data/orig_piv_chart.csv"
inc_piv_df.to_csv(output_data_file, index=True, header=True)

In [25]:
df1.rename(columns = {'disasterNumber':'Disaster'}, inplace = True) 

#fema_mrg = pd.merge(housingasst_df, disaster_df, on='Disaster')
fema_mrg = pd.merge(left=df2,right=df1, left_on='Disaster', right_on='Disaster')

fema_mrg.head(5)
#incidentType, fyDeclared, Disaster,state,incidentType,

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,incidentEndDate,disasterCloseoutDate,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,hash,lastRefresh,address_y
0,4454,TX,Cameron (County),BLUETOWN-IGLESIA ANTIGUA,78559,1,$149,1,$149,0,...,2019-06-25T12:00:00.000Z,NaN,48,0,0,Statewide,19049,NaN,2019-07-17T20:07:32.873Z,"4454, TX, Statewide"
1,4454,TX,Cameron (County),BLUETOWN-IGLESIA ANTIGUA,78559,1,$149,1,$149,0,...,2019-06-25T12:00:00.000Z,NaN,48,215,99215,Hidalgo (County),19049,28b6a4e87b06701c9e2ee664643c9a12,2019-10-10T19:53:15.415Z,"4454, TX, Hidalgo (County)"
2,4454,TX,Cameron (County),BLUETOWN-IGLESIA ANTIGUA,78559,1,$149,1,$149,0,...,2019-06-25T12:00:00.000Z,NaN,48,61,99061,Cameron (County),19049,2d323c9ca51ab1c085f2a9285a74ea46,2019-10-10T19:53:15.414Z,"4454, TX, Cameron (County)"
3,4454,TX,Cameron (County),BLUETOWN-IGLESIA ANTIGUA,78559,1,$149,1,$149,0,...,2019-06-25T12:00:00.000Z,NaN,48,489,99489,Willacy (County),19049,586b7fda30c56e22ccbe418cf0689860,2019-10-10T19:53:15.418Z,"4454, TX, Willacy (County)"
4,4454,TX,Cameron (County),BROWNSVILLE,78520,10,$335,9,"$3,013",2,...,2019-06-25T12:00:00.000Z,NaN,48,0,0,Statewide,19049,NaN,2019-07-17T20:07:32.873Z,"4454, TX, Statewide"


In [26]:
fema_fnl = fema_mrg[["Disaster","incidentType","fyDeclared","state","County", "City", "Zip Code"]]
fema_fnl.dropna(how="any", inplace=True)
fema_fnl.reset_index(drop=True, inplace=True)
fema_no_dups = fema_fnl.drop_duplicates()
len(fema_fnl)

C:\Users\paulk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1481164

In [27]:
abr = fema_fnl[fema_fnl['state'].str.contains('TX')]
fema_tx = abr.drop_duplicates()
fema_tx.head(5)

,Disaster,incidentType,fyDeclared,state,County,City,Zip Code
0,4454,Flood,2019,TX,Cameron (County),BLUETOWN-IGLESIA ANTIGUA,78559
4,4454,Flood,2019,TX,Cameron (County),BROWNSVILLE,78520
8,4454,Flood,2019,TX,Cameron (County),BROWNSVILLE,78521
12,4454,Flood,2019,TX,Cameron (County),BROWNSVILLE,78526
16,4454,Flood,2019,TX,Cameron (County),COMBES,78535


In [28]:
sum_county1 = fema_tx.groupby("County").count().Disaster
sum_incident = fema_tx.groupby("incidentType").count()
sum_fy = fema_tx.groupby("fyDeclared").count()
#um_zip = fema_tx.groupby("Zip Code").count()
print(sum_county1.head(5))
#print(sum_incident.head(5))
#print(sum_fy.head(5))
#rint(sum_zip.head(5))

County
Anderson (County)     6
Angelina (County)     7
Aransas (County)     41
Austin (County)      54
Bastrop (County)     45
Name: Disaster, dtype: int64


In [29]:
#Clean up County Column for FEMA
county_dis = sum_county1.to_frame()
county_dis.reset_index(drop=False, inplace=True)
new = county_dis["County"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
county_dis["Counties"]= new[0] 
  
# making separate last name column from new data frame 
county_dis["type"]= new[1] 
  
# Dropping old Name columns 
county_dis.drop(columns =["County"], inplace = True) 
  
# df display 

county_dis.head(5)

,Disaster,Counties,type
0,6,Anderson,(County)
1,7,Angelina,(County)
2,41,Aransas,(County)
3,54,Austin,(County)
4,45,Bastrop,(County)


In [30]:
#Request API

response4 = requests.get(url4).content

In [31]:
#convert XML to json

resp4 = xmltodict.parse(response4)
json.dumps(resp4)

#convert json to Dataframe
regionchildren_df = pd.DataFrame.from_dict(json_normalize(resp4), orient='columns')


In [32]:
#propertydetails_df.rename(columns=lambda x: x[55:], inplace=True)
#comps_df.rename(columns=lambda x: x[45:], inplace=True)
#searchresults_df.rename(columns=lambda x: x[30:], inplace=True)
regionchildren_df.rename(columns=lambda x: x[30:], inplace=True)
regionchildren_df.head(5)

,@xsi:schemaLocation,@xmlns:xsi,@xmlns:RegionChildren,request.state,request.childtype,message.text,message.code,response.region.id,response.region.latitude,response.region.longitude,response.subregiontype,response.list.count,response.list.region
0,http://www.zillow.com/static/xsd/RegionChildre...,http://www.w3.org/2001/XMLSchema-instance,http://www.zillow.com/static/xsd/RegionChildre...,tx,county,Request successfully processed,0,54,31.170152,-100.076446,county,254,"[{'id': '1090', 'name': 'Harris County', 'lati..."


In [33]:
#extract child(zipcode) which is a dictionary into a dataframe
region_list = pd.DataFrame(regionchildren_df['response.list.region'][0])
#dropna and reset index
region_list.dropna(how="any", inplace=True)
region_list.reset_index(drop=True, inplace=True)

print(region_list.head(5))

     id            name   latitude   longitude  \
0   978   Dallas County  32.767271  -96.777624   
1    98     Bell County  31.036287  -97.492016   
2   498    Ellis County  32.300758  -96.735228   
3  1827  Johnson County  32.344555  -97.351662   
4  1423   Taylor County  32.302102  -99.890743   

                                    zindex  
0  {'@currency': 'USD', '#text': '122000'}  
1  {'@currency': 'USD', '#text': '116600'}  
2  {'@currency': 'USD', '#text': '121300'}  
3  {'@currency': 'USD', '#text': '122300'}  
4   {'@currency': 'USD', '#text': '99800'}  


In [34]:
#extract USD value from dictionary
region_list["USD($)"] = [d.get('#text') for d in region_list.zindex]

region_list

,id,name,latitude,longitude,zindex,USD($)
0,978,Dallas County,32.767271,-96.777624,"{'@currency': 'USD', '#text': '122000'}",122000
1,98,Bell County,31.036287,-97.492016,"{'@currency': 'USD', '#text': '116600'}",116600
2,498,Ellis County,32.300758,-96.735228,"{'@currency': 'USD', '#text': '121300'}",121300
3,1827,Johnson County,32.344555,-97.351662,"{'@currency': 'USD', '#text': '122300'}",122300
4,1423,Taylor County,32.302102,-99.890743,"{'@currency': 'USD', '#text': '99800'}",99800
5,2095,Wichita County,34.023249,-98.688029,"{'@currency': 'USD', '#text': '80900'}",80900
6,1991,Randall County,34.965391,-101.895893,"{'@currency': 'USD', '#text': '137700'}",137700
7,454,Comal County,29.816233,-98.322756,"{'@currency': 'USD', '#text': '237200'}",237200
8,1733,Grayson County,33.678701,-96.662288,"{'@currency': 'USD', '#text': '111700'}",111700
9,1308,Parker County,32.779315,-97.805501,"{'@currency': 'USD', '#text': '174600'}",174600


In [35]:
#Clean up County Column for FEMA


new_2 = region_list["name"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
region_list["Counties"]= new[0] 
  
# making separate last name column from new data frame 
region_list["type"]= new[1] 
  
# Dropping old Name columns 
region_list.drop(columns =["name"], inplace = True) 
  
# df display 

region_list.head(5)

,id,latitude,longitude,zindex,USD($),Counties,type
0,978,32.767271,-96.777624,"{'@currency': 'USD', '#text': '122000'}",122000,Anderson,(County)
1,98,31.036287,-97.492016,"{'@currency': 'USD', '#text': '116600'}",116600,Angelina,(County)
2,498,32.300758,-96.735228,"{'@currency': 'USD', '#text': '121300'}",121300,Aransas,(County)
3,1827,32.344555,-97.351662,"{'@currency': 'USD', '#text': '122300'}",122300,Austin,(County)
4,1423,32.302102,-99.890743,"{'@currency': 'USD', '#text': '99800'}",99800,Bastrop,(County)


In [36]:
region_list["USD($)"] = pd.to_numeric(region_list["USD($)"])

output_region_file = "./interim_data/region_list.csv"
region_list.to_csv(output_region_file, index=True, header=True)

region_list.dtypes

id           object
latitude     object
longitude    object
zindex       object
USD($)        int64
Counties     object
type         object
dtype: object

In [37]:
#merge fema and zillow
fema_zillow_mrg = pd.merge(left=region_list,right=county_dis, left_on='Counties', right_on='Counties')
fema_zillow_mrg["latitude"] = fema_zillow_mrg["latitude"].astype(float)
fema_zillow_mrg["longitude"] = fema_zillow_mrg["longitude"].astype(float)
fema_zillow_mrg.head()

output_zillow_file = "./interim_data/fema_zillow_mrg.csv"
fema_zillow_mrg.to_csv(output_zillow_file, index=True, header=True)